# Paso 2 — Limpieza y Transformación de Datos

**Objetivo:**  
Realizar un proceso completo de limpieza, imputación y transformación de los datos del dataset combinado, preparándolo para el análisis exploratorio y el modelado predictivo.

**Archivo de entrada:** `Telecom_Customer_Churn_Complete.csv`  
**Archivo de salida:** `Telecom_Customer_Churn_Clean.csv`


In [110]:
import pandas as pd
import numpy as np

df = pd.read_csv("..\data\Telecom_Customer_Churn_Complete.csv")

print(f"Dataset cargado: {df.shape[0]} filas, {df.shape[1]} columnas")
df.head()


Dataset cargado: 7043 filas, 65 columnas


<>:4: SyntaxWarning: "\d" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\d"? A raw string is also an option.
<>:4: SyntaxWarning: "\d" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\d"? A raw string is also an option.
C:\Users\Yeison\AppData\Local\Temp\ipykernel_26392\770444338.py:4: SyntaxWarning: "\d" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\d"? A raw string is also an option.
  df = pd.read_csv("..\data\Telecom_Customer_Churn_Complete.csv")


,Customer ID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Unlimited Data,Monthly Charge,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Status ID,Satisfaction Score,Customer Status,Churn Category
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,Yes,53.85,0.0,0,20.94,129.09,SUDNGT6444,1,Churned,Competitor
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,Yes,70.70,0.0,0,18.24,169.89,KZSZDV8891,2,Churned,Other
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Yes,99.65,0.0,0,97.20,917.70,EPTIUU1269,3,Churned,Other
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,...,Yes,104.80,0.0,0,136.92,3182.97,PAJIVH8196,3,Churned,Other
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,...,Yes,103.70,0.0,0,2172.17,7208.47,RXFOMV1173,1,Churned,Competitor


### Limpieza de valores nulos

A continuación se muestran los valores nulos presentes en el dataset

In [111]:
# Mostrar cantidad de nulos por columna
df.isnull().sum().sort_values(ascending=False).head(10)


Churn Reason      5174
Churn Category    5174
Offer             3877
Internet Type     1526
Customer ID          0
Zip Code             0
Count                0
State                0
Country              0
Gender               0
dtype: int64

Como se pudo observar, solo 5 columnas presentan valores nulos, mientras que las demas no. A continuacion se hace el manejo de dichos valores nulos, para el caso de las columnas *Churn Reason*, *Churn Category* y *Churn Reason_Telco_customer_churn_status* se reemplazará dichos valores nulos con "Not Applicable", para las demas columnas se analizara su tipo y se buscara la manera correcta de hacerlo. En el caso de la columna offer que indica que oferta recibio el cliente. se reemplazaran los valores nulos con "No Offer" y en el caso de internet Type se reemplazaran con "No Internet Service".


In [112]:
containing_null = ['Churn Reason', 'Churn Category', 'Offer', 'Internet Type']

df[containing_null].head(20)



,Churn Reason,Churn Category,Offer,Internet Type
0,Competitor made better offer,Competitor,NaN,DSL
1,Moved,Other,NaN,Fiber Optic
2,Moved,Other,NaN,Cable
3,Moved,Other,Offer C,Fiber Optic
4,Competitor had better devices,Competitor,NaN,Fiber Optic
5,Competitor offered higher download speeds,Competitor,NaN,Cable
6,Competitor offered more data,Competitor,NaN,DSL
7,Competitor made better offer,Competitor,NaN,NaN
8,Competitor had better devices,Competitor,NaN,Fiber Optic
9,Competitor had better devices,Competitor,NaN,DSL


In [113]:

df["Offer"] = df["Offer"].fillna("No Offer")
df["Internet Type"] = df["Internet Type"].fillna("No Internet Service")
df["Churn Category"] = df["Churn Category"].fillna("Not Applicable")
df["Churn Reason"] = df["Churn Reason"].fillna("Not Applicable")

# Confirmar cambios
df[["Offer", "Internet Type", "Churn Category", "Churn Reason"]].isnull().sum()


Offer             0
Internet Type     0
Churn Category    0
Churn Reason      0
dtype: int64

### Correccion del formato de Total Charges

Se corregira el formato de la columna Total Charges para que sea de tipo numerico. Se reemplazaran los espacios vacios con 0.0

In [114]:
# Revisar tipo y valores problemáticos
print(df["Total Charges"].dtype)
print(df["Total Charges"].unique()[:10])

# Reemplazar espacios vacíos o strings vacíos por 0
df["Total Charges"] = df["Total Charges"].replace(" ", 0)
df["Total Charges"] = df["Total Charges"].astype(float)


object
['108.15' '151.65' '820.5' '3046.05' '5036.3' '528.35' '39.65' '20.15'
 '4749.15' '30.2']


### Correcion de formatos

En las columnas Online Security, Online Backup, Device Protection, Tech Support, streaming TV y streaming Movies se sustuira el valor de "No internet service" a "No"

En la columna Multiple lines se sustituira los valores de "No phone Service" a "No"

In [115]:
cols_internet = ["Online Security", "Online Backup", "Device Protection", 
                 "Tech Support", "Streaming TV", "Streaming Movies"]

for col in cols_internet:
    df[col] = df[col].replace("No internet service", "No")

df["Multiple Lines"] = df["Multiple Lines"].replace("No phone service", "No")


Se sustituiran los valores de Tenure para reemplazarlos por un rango o categoria de acuerdo a la duración asi:
0–12 Month, 12–24 Month, 24–48 Months, 48–60 Month y > 60 Month

In [116]:
def tenure_group(tenure):
    if tenure <= 12:
        return "0-12 Months"
    elif tenure <= 24:
        return "12-24 Months"
    elif tenure <= 48:
        return "24-48 Months"
    elif tenure <= 60:
        return "48-60 Months"
    else:
        return "> 60 Months"

df["Tenure Group"] = df["Tenure in Months"].apply(tenure_group)


Ahora se corregira el formato de la columna Population para que sea de tipo numerico

In [117]:
df["Population"] = df["Population"].replace(",", "", regex=True).astype(float)


### Eliminación de columnas innecesarias y prevención de data leakage

Durante el proceso de depuración del dataset, se identificaron varias columnas que no aportan valor analítico, presentan información redundante o pueden provocar fugas de datos (*data leakage*). A continuación se detalla el análisis y las decisiones tomadas:

#### 1️⃣ Detección de columnas sin variabilidad
Antes de eliminar columnas específicas, se verificó si existían variables con un único valor en todas las filas, ya que estas no aportan información relevante para el análisis.

#### 2️⃣ Eliminación de columnas redundantes o no analíticas
Se eliminaron las siguientes columnas por las razones descritas:
- **Tenure Months** y **Tenure in Months:** contienen la misma información; se conserva una sola versión.
- **Monthly Charge:** duplicada respecto a **Monthly Charges**.
- **Count:** solo cuenta las filas; no aporta valor analítico.
- **Customer ID, ID, LoyaltyID, Location ID, Service ID, Status ID:** identificadores técnicos sin relevancia analítica.
- **Latitude, Longitude:** información ya contenida en **Lat Long**.
- **Tenure in Months:** resumida en otra variable más representativa.

#### 3️⃣ Depuración de columnas de ubicación
Tras analizar la relevancia de las variables geográficas (`Country`, `State`, `City`, `Zip Code`, `Lat Long`), se observó que:
- Todos los registros pertenecen al mismo país y estado.
- `Zip Code` representa de forma única la localización del cliente.
Por lo tanto, se conservará únicamente **Zip Code** y se eliminarán las demás columnas de ubicación.

#### 4️⃣ Prevención de *data leakage*
Para evitar fugas de información que puedan afectar un modelo predictivo:
- Se eliminaron **Churn Reason** y **Churn Category**, ya que solo contienen información de clientes que ya han cancelado.
- Se identificó que **Churn Value**, **Churn Label** y **Customer Status** representan la misma información que la variable objetivo **Churn**.  
  Por tanto, se conservó únicamente la columna **Churn**.

#### 5️⃣ Columnas sin variabilidad
Durante la revisión, se detectó que la columna **Quarter** tenía un único valor para todas las filas, por lo que fue eliminada por no aportar información al análisis.

Con estas eliminaciones, el dataset queda optimizado, libre de redundancias y sin riesgo de fugas de datos, garantizando su idoneidad para el análisis exploratorio y el modelado posterior.


In [118]:
cols_to_drop = []
#Eliminacion de columnas sin variabilidad
unique_values = df.nunique()
single_value_cols = unique_values[unique_values == 1].index.tolist()
unique_values = df.nunique()
single_value_cols = unique_values[unique_values == 1].index.tolist()
cols_to_drop.extend(single_value_cols)
print("\n Columnas con un único valor:")
print(single_value_cols if single_value_cols else "Ninguna encontrada")


 Columnas con un único valor:
['Count', 'Country', 'State', 'Quarter']


Las columnas sin variabilidad son: Count, Country, State, Quarter. Estas seran añadidas a las que necesitan ser eliminadas.

In [119]:
# Eliminacion de columnas redundantes y sin importancia analitica

redundant_columns = [
    "Tenure in Months",
    "Monthly Charge",
    "Customer ID",
    "ID",
    "LoyaltyID",
    "Location ID",
    "Service ID",
    "Status ID",
    "Latitude",
    "Longitude",
    "Tenure Months",
    "City",
    "Lat Long"
]



leakage_cols = ["Churn Reason", "Churn Category", "Churn Value", "Churn Label", "Customer Status"]



cols_to_drop.extend(redundant_columns)
cols_to_drop.extend(leakage_cols)
cols_to_drop = list(set(cols_to_drop))
print(f"\n Columnas a eliminar ({len(cols_to_drop)}): {cols_to_drop}")
# Eliminar las columnas identificadas
df = df.drop(columns=cols_to_drop)

# Mostrar el resultado final
print(f"\n Dataset actualizado: {df.shape[1]} columnas restantes.")



 Columnas a eliminar (22): ['Churn Category', 'Status ID', 'City', 'State', 'Count', 'Country', 'Lat Long', 'Latitude', 'ID', 'Tenure in Months', 'LoyaltyID', 'Churn Label', 'Customer Status', 'Monthly Charge', 'Customer ID', 'Churn Value', 'Longitude', 'Service ID', 'Tenure Months', 'Location ID', 'Churn Reason', 'Quarter']

 Dataset actualizado: 44 columnas restantes.


In [120]:
OUTPUT_PATH = "../data/Telecom_Customer_Churn_Complete.csv"

df.to_csv(OUTPUT_PATH, index=False)

print(f"✅ Archivo final guardado en: {OUTPUT_PATH}")

✅ Archivo final guardado en: ../data/Telecom_Customer_Churn_Complete.csv
